In [155]:
import nltk
import os
import numpy as np
import pandas as pd
from multiprocessing.pool import ThreadPool
import time



In [108]:
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

In [109]:
userhome = os.path.expanduser('~')
csvfile= userhome + r'/Desktop/cd_mumbai_complaints.csv'
df = pd.read_csv(csvfile)




In [110]:
df

,generic_id,created_at,complaint_status_id,voteup_count,city_id,ward_id,category_id,sub_category_id,civic_agency_id,title,description,location
0,W01730C147452,We are facing multiple issues related to road ...,4,28,2,946,15.0,66,22.0,8/5/16 9:45,"Potholes, Illegal Parking and Movement of Heav...","NSS Palkar Road, Asalfa Village, Ghatkopar Wes..."
1,W0960C153580,This is a really bad road. we need it cleaned ...,4,2,2,943,6.0,155,22.0,5/6/16 17:11,Bad road now with garbage,"21st Rd, MIDC Industrial Estate, Pandit Dinday..."
2,W01760C153649,This needs to be fixed asap.,2,40,2,946,21.0,84,25.0,5/6/16 21:16,Traffic issue that needs immediate attention,"5, JVLR, IIT Area, Powai, Mumbai, Maharashtra ..."
3,W01590C157072,We observed frequently heavy traffic at dahisa...,2,7,2,933,21.0,84,25.0,16-06-2016 19:42,Traffic jam at Dahisar check naka,"Western Express Hwy, Diamond Industrial Estate..."
4,W02080C164464,A pothole in front of Chand Shahwli Baba Darga...,3,19079,2,945,15.0,66,22.0,13-07-2016 12:57,Pothole on Pipeline Road,"Pipe Line Rd, Powai, IIT Area, Powai, Mumbai, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9293,W09460C398356,Garbage on foot path,1,0,2,946,6.0,155,22.0,8/9/18 11:11,Garbage on foot path,"Wing-B, Chandivali Rd, Yadav Nagar, Chandivali..."
9294,W09460C398357,Garbage near foot path,1,0,2,946,6.0,155,22.0,8/9/18 11:12,Garbage near foot path,"Manubhai Seetal Chawl, Chandivali Rd, Yadav Na..."
9295,W09550C398400,Please add sound barrier Along all the suburba...,1,0,2,955,313.0,314,28.0,8/9/18 12:37,Please add sound barrier Along all the suburba...,"Planet Godrej, Jacob Circle, Mumbai, Maharasht..."
9296,W09440C398666,Garbage on foot path,1,0,2,944,6.0,155,22.0,9/9/18 8:31,Garbage on foot path,"D-2, Pandit Madan Mohan Malaviya Rd, Mtnl Colo..."


In [111]:
import nltk
from nltk.tokenize import word_tokenize

In [112]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jarvis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [113]:
df1 = df[pd.notnull(df['created_at'])]
df1 = df1[pd.notnull(df['category_id'])]
df1 = df1[pd.notnull(df['civic_agency_id'])]






/Users/jarvis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/jarvis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [114]:
df1.reset_index(inplace=True)

In [115]:
import nltk
from nltk.corpus import stopwords

In [116]:
for i in range(len(df1)):
    text = df1["created_at"][i] 
    df1["created_at"][i] = ' '.join([word for word in text.split() if word not in stopwords.words("english")])

/Users/jarvis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [117]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
from sklearn.feature_extraction.text import CountVectorizer
cvectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()



In [134]:
xt = vectorizer.fit_transform(df1["created_at"])
xc = cvectorizer.fit_transform(df1["created_at"])

In [135]:
len(vectorizer.vocabulary_) 

6964

In [139]:
len(cvectorizer.vocabulary_) 

6964

In [140]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [158]:
from sklearn.model_selection import train_test_split

In [159]:
train_vectors, test_vectors, y_train, y_test = train_test_split( xt, df1["sub_category_id"] , test_size=0.2, random_state=42)

In [160]:
start = time.time()
model1 = XGBClassifier()
model1.fit(train_vectors, y_train)
print(model1)
y_pred = model1.predict(test_vectors)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
end = time.time()

print(f"Runtime of the program is {end - start}")


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)
Accuracy: 90.83%
Runtime of the program is 133.03442287445068


In [161]:
train_vectors, test_vectors, y_train, y_test = train_test_split( xc, df1["sub_category_id"] , test_size=0.2, random_state=42)

In [162]:
start = time.time()
model1a = XGBClassifier()
model1a.fit(train_vectors, y_train)
print(model1a)
y_pred = model1a.predict(test_vectors)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
end = time.time()

print(f"Runtime of the program is {end - start}")

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)
Accuracy: 90.95%
Runtime of the program is 128.7261188030243


In [163]:
train_vectors , test_vectors, y_train, y_test = train_test_split( xt , df1["category_id"] , test_size=0.2, random_state=42)

In [164]:
start = time.time()
model2 = XGBClassifier()
model2.fit(train_vectors, y_train)
print(model2)
y_pred = model2.predict(test_vectors)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
end = time.time()

print(f"Runtime of the program is {end - start}")

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)
Accuracy: 93.23%
Runtime of the program is 27.6805100440979


In [165]:
train_vectors , test_vectors, y_train, y_test = train_test_split( xc , df1["category_id"] , test_size=0.2, random_state=42)

In [166]:
start = time.time()
model2a = XGBClassifier()
model2a.fit(train_vectors, y_train)
print(model2a)
y_pred = model2a.predict(test_vectors)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
end = time.time()

print(f"Runtime of the program is {end - start}")

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)
Accuracy: 93.65%
Runtime of the program is 24.47632908821106


In [167]:
train_vectors , test_vectors, y_train, y_test = train_test_split( xt , df1["civic_agency_id"] , test_size=0.2, random_state=42)

In [168]:
start = time.time()
model3 = XGBClassifier()
model3.fit(train_vectors, y_train)
print(model3)
y_pred = model3.predict(test_vectors)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
end = time.time()

print(f"Runtime of the program is {end - start}")

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)
Accuracy: 96.28%
Runtime of the program is 11.171482801437378


In [169]:
train_vectors , test_vectors, y_train, y_test = train_test_split( xc , df1["civic_agency_id"] , test_size=0.2, random_state=42)

In [170]:
start = time.time()
model3a = XGBClassifier()
model3a.fit(train_vectors, y_train)
print(model3a)
y_pred = model3a.predict(test_vectors)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
end = time.time()

print(f"Runtime of the program is {end - start}")

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)
Accuracy: 96.28%
Runtime of the program is 8.206562042236328
